In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import numpy as np
import pandas as pd

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.dates import YearLocator, MonthLocator
plt.rcParams["figure.figsize"] = (12, 8) # (width, height)

In [3]:
strategy = os.getcwd()
print("change working directory to parent")
os.chdir('/mnt/c/workspace/project_quant')
print(os.getcwd())

change working directory to parent
/mnt/c/workspace/project_quant


In [6]:
from utils.index_data import index_data
import pandas_datareader.data as web

In [18]:
data_setting = {
'start' : '1992-01-01',
'end'   : '2021-06-30',
'rebal_period':'1M'
}

list_index_data = ['msci_world', 'msci_emerging', 'msci_world_gross','msci_world_value', 'bb_world_agg', 'bb_corp_ig', 'bb_corp_hy']
macro_index = [
    'T10Y2Y' , # US 10 year treasury yield - 2 year treasury
    'VIXCLS',    # VIX data
    ]
list_market_data = list_index_data + macro_index

data = index_data(**data_setting)
df = data.index_cum_data_loading()
df = df[list_index_data]

df_macro_data = web.DataReader(macro_index, 'fred', pd.to_datetime('1992-01-01'), pd.to_datetime('2021-06-30'))
df_macro_data.columns = ['yield_curve', 'vix_index']
df_macro_data = df_macro_data.pct_change().fillna(0)

market_data = df.join(df_macro_data, how='inner')

In [24]:
# code adapted from http://hmmlearn.readthedocs.io
# for sklearn 18.1

col = 'sret'
select = market_data.ix[:].dropna()

ft_cols = list_market_data + ['sret']
X = select[ft_cols].values

model = mix.GaussianMixture(n_components=3, 
                            covariance_type="full", 
                            n_init=100, 
                            random_state=7).fit(X)

# Predict the optimal sequence of internal hidden state
hidden_states = model.predict(X)

print("Means and vars of each hidden state")
for i in range(model.n_components):
    print("{0}th hidden state".format(i))
    print("mean = ", model.means_[i])
    print("var = ", np.diag(model.covariances_[i]))
    print()

sns.set(font_scale=1.25)
style_kwds = {'xtick.major.size': 3, 'ytick.major.size': 3,
              'font.family':u'courier prime code', 'legend.frameon': True}
sns.set_style('white', style_kwds)

fig, axs = plt.subplots(model.n_components, sharex=True, sharey=True, figsize=(12,9))
colors = cm.rainbow(np.linspace(0, 1, model.n_components))

for i, (ax, color) in enumerate(zip(axs, colors)):
    # Use fancy indexing to plot data in each state.
    mask = hidden_states == i
    ax.plot_date(select.index.values[mask],
                 select[col].values[mask],
                 ".-", c=color)
    ax.set_title("{0}th hidden state".format(i), fontsize=16, fontweight='demi')

    # Format the ticks.
    ax.xaxis.set_major_locator(YearLocator())
    ax.xaxis.set_minor_locator(MonthLocator())
    sns.despine(offset=10)

plt.tight_layout()
fig.savefig('Hidden Markov (Mixture) Model_Regime Subplots.png')

,msci_world,msci_emerging,msci_world_gross,msci_world_value,bb_world_agg,bb_corp_ig,bb_corp_hy,yield_curve,vix_index
1992-01-31,1.000000,1.000000,0.968536,0.990900,0.997829,0.997412,1.035225,2.20,17.40
1992-03-31,0.962324,1.163400,0.899575,0.928497,0.987790,0.991802,1.073920,1.94,16.18
1992-04-30,0.898778,1.198026,0.889040,0.961648,0.991594,0.997377,1.077999,2.15,15.53
1992-06-30,0.958458,1.186634,0.891546,0.960478,1.010517,1.016692,1.103503,2.31,13.35
1992-07-31,0.935165,1.069737,0.898809,0.954368,1.028941,1.037883,1.120163,2.30,13.17
...,...,...,...,...,...,...,...,...,...
2020-12-31,4.873375,4.238227,6.158531,3.620125,4.740902,6.135291,8.765545,0.80,22.75
2021-03-31,5.199679,4.717694,6.507391,3.898726,4.653979,5.981918,8.998323,1.58,19.40
2021-04-30,5.305197,4.623407,6.500032,4.057982,4.619002,5.934048,9.004689,1.49,18.61
2021-05-31,5.507827,4.636046,6.788088,4.190198,4.642625,5.970003,9.096146,NaN,NaN


In [ ]:
sns.set(font_scale=1.5)
states = (pd.DataFrame(hidden_states, columns=['states'], index=select.index)
          .join(select, how='inner')
          .assign(mkt_cret=select.sret.cumsum())
          .reset_index(drop=False)
          .rename(columns={'index':'Date'}))
p(states.head())

sns.set_style('white', style_kwds)
order = [0, 1, 2]
fg = sns.FacetGrid(data=states, hue='states', hue_order=order,
                   palette=scolor, aspect=1.31, size=12)
fg.map(plt.scatter, 'Date', mkt, alpha=0.8).add_legend()
sns.despine(offset=10)
fg.fig.suptitle('Historical SPY Regimes', fontsize=24, fontweight='demi')
fg.savefig('Hidden Markov (Mixture) Model_SPY Regimes.png')

https://datascienceschool.net/02%20mathematics/03.05%20PCA.html

In [48]:
data_setting = {
'start' : '1992-01-01',
'end'   : '2021-06-30',
'rebal_period':'1M'
}

data = index_data(**data_setting)
df = data.index_cum_data_loading()
df = df[['msci_world', 'msci_emerging', 'msci_world_gross','msci_world_value',
'bb_world_agg', 'bb_corp_ig', 'bb_corp_hy']].T

In [49]:
df

,1992-01-31,1992-02-29,1992-03-31,1992-04-30,1992-05-31,1992-06-30,1992-07-31,1992-08-31,1992-09-30,1992-10-31,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
msci_world,1.000000,0.979322,0.962324,0.898778,0.923565,0.958458,0.935165,0.925919,0.946223,0.932120,...,4.608581,4.445644,4.337530,4.873375,5.024731,5.039375,5.199679,5.305197,5.507827,5.570121
msci_emerging,1.000000,1.103012,1.163400,1.198026,1.186704,1.186634,1.069737,1.056198,1.019114,1.021849,...,3.878186,3.756337,3.859972,4.238227,4.472819,4.712916,4.717694,4.623407,4.636046,4.815963
msci_world_gross,0.968536,0.945533,0.899575,0.889040,0.926601,0.891546,0.898809,0.930289,0.920312,0.898921,...,5.974730,5.750048,5.543643,6.158531,6.397756,6.460124,6.507391,6.500032,6.788088,6.783666
msci_world_value,0.990900,0.976859,0.928497,0.961648,0.993760,0.960478,0.954368,0.964119,0.953588,0.920567,...,3.323973,3.215419,3.181097,3.620125,3.705148,3.690848,3.898726,4.057982,4.190198,4.284581
bb_world_agg,0.997829,0.984762,0.987790,0.991594,0.998068,1.010517,1.028941,1.048780,1.061627,1.076785,...,4.744209,4.734309,4.715387,4.740902,4.761737,4.732557,4.653979,4.619002,4.642625,4.648800
bb_corp_ig,0.997412,0.985340,0.991802,0.997377,1.001546,1.016692,1.037883,1.063868,1.074932,1.091555,...,6.040019,6.008772,6.000955,6.135291,6.172184,6.108264,5.981918,5.934048,5.970003,5.996821
bb_corp_hy,1.035225,1.060768,1.073920,1.077999,1.093210,1.103503,1.120163,1.134840,1.146392,1.130228,...,8.467538,8.387214,8.434219,8.765545,8.912584,8.949182,8.998323,9.004689,9.096146,9.120544
